In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Kannada-MNIST/test.csv
/kaggle/input/Kannada-MNIST/Dig-MNIST.csv
/kaggle/input/Kannada-MNIST/sample_submission.csv
/kaggle/input/Kannada-MNIST/train.csv


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
traind = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
testd = pd.read_csv('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv')

In [5]:
labels = traind.iloc[:,:1]
label_one_hot = pd.get_dummies(traind['label'])
#print((label_one_hot))
#print(label_one_hot.shape)
#torch vectors:
labels = torch.tensor(labels.values)
label1 = torch.tensor(label_one_hot.values)

In [6]:
labely = testd.iloc[:,:1]
#torch vectors:
labely = torch.tensor(labely.values)
labely = labely.reshape(10240)

In [7]:
trainx = torch.from_numpy(traind.iloc[:,1:].values).float()
trainx = trainx.reshape(60000,784,1)
print('trainx: ',trainx.shape)

trainx:  torch.Size([60000, 784, 1])


In [8]:
labels = labels.reshape(60000)
print(labels.shape)

torch.Size([60000])


In [24]:
labels.shape

torch.Size([60000, 1])

In [9]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
               
        self.conv1 = nn.Conv1d(784,512,1)
        self.conv2 = nn.Conv1d(512,128,1)
        self.conv3 = nn.Conv1d(128,64,1)
        self.conv4 = nn.Conv1d(64,16,1)
        self.conv5 = nn.Conv1d(16,10,1)
        self.conv6 = nn.Conv1d(1,1,1)
        
        self.max1 = nn.MaxPool1d(1,3)
        self.max2 = nn.MaxPool1d(1,7)
        self.avg1 = nn.AvgPool1d(1,3)
        self.avg2 = nn.AvgPool1d(1,7)
        
        self.batch1 = nn.BatchNorm1d(512)
        self.batch2 = nn.BatchNorm1d(128)
        self.batch3 = nn.BatchNorm1d(64)
        self.batch4 = nn.BatchNorm1d(16)
        
        self.soft = nn.LogSoftmax(dim=1)
    def forward(self, tx):
        x = self.conv1(tx)
        #print(x.shape)
        x = self.max1(x)
        #print(x.shape)
        x = self.batch1(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)
        
        x = self.conv2(x)
        #print(x.shape)
        x = self.max2(x)
        #print(x.shape)
        x = self.batch2(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)

        x = self.conv3(x)
        #print(x.shape)
        x = self.avg1(x)
        #print(x.shape)
        x = self.batch3(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)

        x = self.conv4(x)
        #print(x.shape)
        x = self.avg2(x)
        #print(x.shape)
        x = self.batch4(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)

        x = self.conv5(x)
        #print(x.shape)
        x = self.soft(x)
        #print(x.shape)

        out = x.reshape(tx.shape[0], 10)
        return out
        

In [10]:
model = net()
crit = nn.NLLLoss()
opt = optim.Adam(model.parameters(),lr = 0.09)#0.22

for i in range(1):
    out = model(trainx)
    loss = crit(out,labels)
    print(i,loss.item())
    loss.backward()
    opt.step()
    opt.zero_grad()

0 2.3924431800842285


In [19]:
model = net().to(device)
crit = nn.NLLLoss()
opt = optim.Adam(model.parameters(),lr = 0.09)#0.22

train_losses = []
valid_losses = []

for i in range(66):
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    trainx = trainx.to(device)
    labels = labels.to(device)
    
    opt.zero_grad()
    out = model(trainx)
    loss = crit(out,labels)
    print(i,loss.item(),end = " ")
    loss.backward()
    opt.step()
    
    model.eval()
    testx = testx.to(device)
    labely = labely.to(device)
    
    out2 = model(testx)
    loss2 = crit(out2,labely)
    print(loss2.item())
    

0 2.4182982444763184 103.78511047363281
1 2.3054914474487305 80.17951965332031
2 1.9027971029281616 66.66064453125
3 1.6440725326538086 42.54212188720703
4 1.413390874862671 27.824926376342773
5 1.2177916765213013 20.31528663635254
6 1.049516201019287 15.548977851867676
7 0.8783397674560547 12.795978546142578
8 0.7283703684806824 10.960580825805664
9 0.5956215858459473 9.439802169799805
10 0.4599405527114868 8.828069686889648
11 0.37095746397972107 9.071538925170898
12 0.3036683201789856 9.267646789550781
13 0.26319125294685364 8.145739555358887
14 0.23919591307640076 7.474677085876465
15 0.21743015944957733 7.1408233642578125
16 0.19571098685264587 6.882411003112793
17 0.1812419295310974 6.664430141448975
18 0.1743488907814026 6.494129180908203
19 0.1686413288116455 6.376202583312988
20 0.15974846482276917 6.274486541748047
21 0.1488141417503357 6.139729976654053
22 0.13980256021022797 5.942530155181885
23 0.13344019651412964 5.69042444229126
24 0.12847886979579926 5.417509078979492
2

In [13]:
print(crit(model(testx),labely))

tensor(2.2766, grad_fn=<NllLossBackward>)


In [11]:
testx = torch.from_numpy(testd.iloc[:,1:].values).float()
print(testx.shape)
testx = testx.reshape(10240,784,1)
print('testx: ',testx.shape)


torch.Size([10240, 784])
testx:  torch.Size([10240, 784, 1])


In [12]:
print(testx.shape)
print(labely.shape)
print(labely)

torch.Size([10240, 784, 1])
torch.Size([10240])
tensor([0, 1, 2,  ..., 7, 8, 9])
